# Assignment 1: Wrangling and EDA
### Foundations of Machine Learning

In [1]:
! git clone https://github.com/NickLarsonUVA/scratchpad
%run ./scratchpad/get_data.py

Cloning into 'scratchpad'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 51 (delta 13), reused 6 (delta 6), pack-reused 32 (from 1)
Receiving objects: 100% (51/51), 132.11 KiB | 2.49 MiB/s, done.
Resolving deltas: 100% (22/22), done.
Download complete
Extracting data files...
Data extracted


In [2]:
import seaborn as sns
import numpy as np
import pandas as pd

**Q1.** This question provides some practice cleaning variables which have common problems.
1. Numeric variable: For `airbnb_NYC.csv`, clean the `Price` variable as well as you can, and explain the choices you make. How many missing values do you end up with? (Hint: What happens to the formatting when a price goes over 999 dollars, say from 675 to 1,112?)
2. Categorical variable: For the Minnesota police use of for data, `mn_police_use_of_force.csv`, clean the `subject_injury` variable, handling the NA's; this gives a value `Yes` when a person was injured by police, and `No` when no injury occurred. What proportion of the values are missing? Cross-tabulate your cleaned `subject_injury` variable with the `force_type` variable. Are there any patterns regarding when the data are missing? For the remaining missing values, replace the `np.nan/None` values with the label `Missing`.
3. Dummy variable: For `metabric.csv`, convert the `Overall Survival Status` variable into a dummy/binary variable, taking the value 0 if the patient is deceased and 1 if they are living.
4. Missing values: For `airbnb_NYC.csv`, determine how many missing values of `Review Scores Rating` there are. Create a new variable, in which you impute the median score for non-missing observations to the missing ones. Why might this bias or otherwise negatively impact your results?

In [27]:
#1
df = pd.read_csv('/content/data/airbnb_NYC.csv', encoding='latin1')
#df.head() # once price hits 4 digits it has comma, need to get rid
df['Price_clean'] = (df['Price'].str.replace(',', '', regex=False)) #replace comma
df['Price_clean'] = pd.to_numeric(df['Price_clean'], errors='coerce') #change to numeric
print(df['Price_clean'].describe())
#df['Price_clean'].value_counts()
df['Price_clean'].isna().sum()
# After removing commas and converting the variable to
# numeric with errors='coerce', no values were dropped.


count    30478.000000
mean       163.589737
std        197.785454
min         10.000000
25%         80.000000
50%        125.000000
75%        195.000000
max      10000.000000
Name: Price_clean, dtype: float64


np.int64(0)

In [50]:
#2
df = pd.read_csv('/content/data/mn_police_use_of_force.csv')
#show that there are more missing than Yes / No
df['subject_injury'].value_counts(dropna=False)
df['subject_injury_clean'] = df['subject_injury']
df['subject_injury_clean'].isna().mean()
pd.crosstab(df['force_type'], df['subject_injury_clean'], dropna=False, normalize='index')
#cross tabulating allows to see what type of force has the most missing
#can see patterns with the types: much more common for less lethal force types
#(like Chemical Irritant, Taser, Bodily Force) and rare for clearly defined uses
#(like Firearm) suggesting that missingness is not random and depends on the type of force used.


subject_injury_clean,No,Yes,NaN
force_type,,,
Baton,0.000000,0.500000,0.500000
Bodily Force,0.115907,0.136373,0.747720
Chemical Irritant,0.082235,0.025738,0.892028
Firearm,1.000000,0.000000,0.000000
Gun Point Display,0.317308,0.423077,0.259615
Improvised Weapon,0.229730,0.270270,0.500000
Less Lethal,0.000000,0.000000,1.000000
Less Lethal Projectile,0.333333,0.666667,0.000000
Maximal Restraint Technique,0.000000,0.000000,1.000000


In [51]:
#2 continued
df['subject_injury_final'] = df['subject_injury_clean'].fillna('Missing') # replaces with missing
df['subject_injury_final'].value_counts() # shows new missing values

,count
subject_injury_final,
Missing,9848
Yes,1631
No,1446


In [56]:
#3
df = pd.read_csv('/content/data/metabric.csv')
df['Overall Survival Status'].value_counts() #shows titles for below also
# create mapping dictionary, replaces string with number
df['OS_dummy'] = df['Overall Survival Status'].map({'1:DECEASED': 0, '0:LIVING': 1})
df['OS_dummy'].value_counts()

,count
OS_dummy,
0,768
1,575


In [65]:
#4
df = pd.read_csv('/content/data/airbnb_NYC.csv', encoding='latin1')
df['Review Scores Rating'].describe()
df['Review Scores Rating'].isna().sum() #8323 missing
df['Review Scores Rating_final'] = df['Review Scores Rating'].fillna(df['Review Scores Rating'].median())
# replaces all missing values (NaN) in Review Scores Rating with the median of the non-missing ratings
#df[['Review Scores Rating', 'Review Scores Rating_final']].head(10) #comparing
df[['Review Scores Rating', 'Review Scores Rating_final']].describe()
# can see from this that the values like mean change 91.99 - 92.54, sd changed 8.85 - 7.60
# showing that replacing missing values reduces variability and shifts the distribution upward in this case

,Review Scores Rating,Review Scores Rating_final
count,22155.000000,30478.000000
mean,91.993230,92.541243
std,8.850373,7.598524
min,20.000000,20.000000
25%,89.000000,90.000000
50%,94.000000,94.000000
75%,100.000000,97.000000
max,100.000000,100.000000


**Q2.** Go to https://sharkattackfile.net/ and download their dataset on shark attacks.

1. Open the shark attack file using Pandas. It is probably not a csv file, so `read_csv` won't work. What does work?
2. Drop any columns that do not contain data.
3. What is an observation? Carefully justify your answer, and explain how it affects your choices in cleaning and analyzing the data.
4. Clean the year variable. Describe the range of values you see. Filter the rows to focus on attacks since 1940. Are attacks increasing, decreasing, or remaining constant over time?
5. Clean the Age variable and make a histogram of the ages of the victims.
6. Clean the `Type` variable so it only takes three values: Provoked and Unprovoked and Unknown. What proportion of attacks are unprovoked?
7. Clean the `Fatal Y/N` variable so it only takes three values: Y, N, and Unknown.
8. Is the attack more or less likely to be fatal when the attack is provoked or unprovoked? Thoughts?

**Q3.** Open the "tidy_data.pdf" document available in `https://github.com/ds4e/wrangling`, which is a paper called *Tidy Data* by Hadley Wickham.

  1. Read the abstract. What is this paper about?
  2. Read the introduction. What is the "tidy data standard" intended to accomplish?
  3. Read the intro to section 2. What does this sentence mean: "Like families, tidy datasets are all alike but every messy dataset is messy in its own way." What does this sentence mean: "For a given dataset, it’s usually easy to figure out what are observations and what are variables, but it is surprisingly difficult to precisely define variables and observations in general."
  4. Read Section 2.2. How does Wickham define values, variables, and observations?
  5. How is "Tidy Data" defined in section 2.3?
  6. Read the intro to Section 3 and Section 3.1. What are the 5 most common problems with messy datasets? Why are the data in Table 4 messy? What is "melting" a dataset?
  7. Why, specifically, is table 11 messy but table 12 tidy and "molten"?

**Q4.** This question looks at financial transfers from international actors to American universities. In particular, from which countries and giftors are the gifts coming from, and to which institutions are they going?

For this question, `.groupby([vars]).count()` and `.groupby([vars]).sum()` will be especially useful to tally the number of occurrences and sum the values of those occurrences.

1. Load the `ForeignGifts_edu.csv` dataset.
2. For `Foreign Gift Amount`, create a histogram and describe the variable. Describe your findings.
3. For `Gift Type`, create a histogram or value counts table. What proportion of the gifts are contracts, real estate, and monetary gifts?
4. What are the top 15 countries in terms of the number of gifts? What are the top 15 countries in terms of the amount given?
5. What are the top 15 institutions in terms of the total amount of money they receive? Make a histogram of the total amount received by all institutions.
6. Which giftors provide the most money, in total?

**Q5.** This question provides some practice doing exploratory data analysis and visualization.

We'll use the `college_completion.csv` dataset from the US Department of Education. The "relevant" variables for this question are:
  - `level` - Level of institution (4-year, 2-year)
  - `aid_value` - The average amount of student aid going to undergraduate recipients
  - `control` - Public, Private not-for-profit, Private for-profit
  - `grad_100_value` - percentage of first-time, full-time, degree-seeking undergraduates who complete a degree or certificate program within 100 percent of expected time (bachelor's-seeking group at 4-year institutions)

1. Load the `college_completion.csv` data with Pandas.
2. How many observations and variables are in the data? Use `.head()` to examine the first few rows of data.
3. Cross tabulate `control` and `level`. Describe the patterns you see in words.
4. For `grad_100_value`, create a kernel density plot and describe table. Now condition on `control`, and produce a kernel density plot and describe tables for each type of institutional control. Which type of institution appear to have the most favorable graduation rates?
5. Make a scatterplot of `grad_100_value` by `aid_value`, and compute the covariance and correlation between the two variables. Describe what you see. Now make the same plot and statistics, but conditioning on `control`. Describe what you see. For which kinds of institutions does aid seem to vary positively with graduation rates?

**Q6.** In class, we talked about how to compute the sample mean of a variable $X$,
$$
m(X) = \dfrac{1}{N} \sum_{i=1}^N x_i
$$
and sample covariance of two variables $X$ and $Y$,
$$
\text{cov}(X,Y) = \dfrac{1}{N} \sum_{i=1}^N (x_i - m(X))(y_i - m(Y))).
$$
Recall, the sample variance of $X$ is
$$
s^2 = \dfrac{1}{N} \sum_{i=1}^N (x_i - m(X))^2.
$$
It can be very helpful to understand some basic properties of these statistics. If you want to write your calculations on a piece of paper, take a photo, and upload that to your GitHub repo, that's probably easiest.

We're going to look at **linear transformations** of $X$, $Y = a + bX$. So we take each value of $X$, $x_i$, and transform it as $y_i = a + b x_i$.

1. Show that $m(a + bX) = a+b \times m(X)$.
2. Show that $ \text{cov}(X,X) = s^2$.
3. Show that $\text{cov}(X,a+bY) = b \times \text{cov}(X,Y)$
4. Show that $\text{cov}(a+bX,a+bY) = b^2 \text{cov}(X,Y) $. Notice, this also means that $\text{cov}(bX, bX) = b^2 s^2$.
5. Suppose $b>0$ and let the median of $X$ be $\text{med}(X)$. Is it true that the median of $a+bX$ is equal to $a + b \times \text{med}(X)$? Is the IQR of $a + bX$ equal to $a + b \times \text{IQR}(X)$?
6. Show by example that the means of $X^2$ and $\sqrt{X}$ are generally not $(m(X))^2$ and $\sqrt{m(X)}$. So, the results we derived above really depend on the linearity of the transformation $Y = a + bX$, and transformations like $Y = X^2$ or $Y = \sqrt{X}$ will not behave in a similar way.

**Q7.** This question provides some practice doing exploratory data analysis and visualization.

We'll use the `ames_prices.csv` dataset. The "relevant" variables for this question are:
  - `price` - Sale price value of the house
  - `Bldg.Type` - Building type of the house (single family home, end-of-unit townhome, duplex, interior townhome, two-family conversion)

1. Load the `college_completion.csv` data with Pandas.
2. Make a kernel density plot of price and compute a describe table. Now, make a kernel density plot of price conditional on building type, and use `.groupby()` to make a describe type for each type of building. Which building types are the most expensive, on average? Which have the highest variance in transaction prices?
3. Make an ECDF plot of price, and compute the sample minimum, .25 quantile, median, .75 quantile, and sample maximum (i.e. a 5-number summary).
4. Make a boxplot of price. Are there outliers? Make a boxplot of price conditional on building type. What patterns do you see?
5. Make a dummy variable indicating that an observation is an outlier.
6. Winsorize the price variable, and compute a new kernel density plot and describe table. How do the results change?